# HR Analytics – Environment

### Outline (Ideas to investigate)

- Is the company diverse?

- Which features can be used to predict employee turnover?

- Which features are linked to a bad working environment?

- Is there any particular department where the environment is worse?

- Any managers in particular cause a bad working environment?

### Import Relevant Packages

In [1]:
import math
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score,GridSearchCV,train_test_split,KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.svm import LinearSVC
from collections import Counter
np.set_printoptions(suppress=True)
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix, precision_recall_curve, PrecisionRecallDisplay
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import precision_score, f1_score, recall_score

## Import Data / Data Cleaning

In [2]:
# Add column 'VoluntaryLeft' 
df = pd.read_csv('BS Tech Case data.csv', sep=';')
df['*VoluntaryLeft'] = (df['EmploymentStatus']=='Voluntarily Terminated').astype(int)

In [3]:
# Add column 'LastWorkingDate' -> Used later to calculate MonthsSpent in the company
df['*LastWorkingDate']=df['DateofTermination'].replace(np.nan, '27/2/2023')
df[['DateofHire','DateofTermination','*LastWorkingDate']] = df[['DateofHire','DateofTermination','*LastWorkingDate']].apply(pd.to_datetime)

In [4]:
# Add column 'MonthsSpent' -> Shows the number of months that the worker has been part of the company
df['*MonthsSpent'] = round((df['*LastWorkingDate']-df['DateofHire'])/np.timedelta64(1, 'M'),0)

In [5]:
# Column 'EngagementSurvey': changed commas or decimal points & converted to float64
for i in range(len(df)):
    df['EngagementSurvey'].iloc[i]= df['EngagementSurvey'].iloc[i].replace(',','.')
df['EngagementSurvey'] = df['EngagementSurvey'].astype(float)

/shared-libs/python3.9/py/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [6]:
df

,Employee.Name,PerfScore,FromDiversityJobFair,Salary,Termd,PositionID,Position,State,Zip,DOB,...,OrgIden,ConHour,CarOpp,PsySafe,Voice,Trust,OCB,*VoluntaryLeft,*LastWorkingDate,*MonthsSpent
0,"Adinolfi, Wilson K",3,0,62506,0,19,Production Technician I,MA,1960,07/10/83,...,3,32,3,4,5,5,4,0,2023-02-27,140.0
1,"Ait Sidi, Karthikeyan",4,0,104437,1,27,Sr. DBA,MA,2148,05/05/75,...,3,40,1,2,3,3,3,1,2020-06-16,63.0
2,"Akinkuolie, Sarah",2,0,64955,1,20,Production Technician II,MA,1810,09/19/88,...,2,40,1,2,2,2,1,1,2016-09-24,63.0
3,"Alagbe,Trina",4,0,64991,0,19,Production Technician I,MA,1886,09/27/88,...,3,40,4,3,4,4,4,0,2023-02-27,182.0
4,"Anderson, Carol",3,0,50825,1,19,Production Technician I,MA,2169,09/08/89,...,3,36,3,2,2,4,3,1,2020-09-06,110.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,"Woodson, Jason",3,0,65893,0,20,Production Technician II,MA,1810,05/11/85,...,2,40,2,2,3,3,3,0,2023-02-27,104.0
307,"Ybarra, Catherine",3,0,48513,1,19,Production Technician I,MA,2458,05/04/82,...,2,40,1,1,1,3,1,1,2019-09-29,133.0
308,"Zamora, Jennifer",5,0,220450,0,6,CIO,MA,2067,08/30/79,...,4,40,3,3,5,4,3,0,2023-02-27,155.0
309,"Zhou, Julia",4,0,89292,0,9,Data Analyst,MA,2148,02/24/79,...,4,40,4,1,2,3,3,0,2023-02-27,95.0


In [7]:
df['Termd'].value_counts()

0    207
1    104
Name: Termd, dtype: int64

In [8]:
df_dummies = pd.get_dummies(df, prefix=['Department','RaceDesc','ManagerID'], columns=['Department','RaceDesc','ManagerID'])

In [9]:
df.describe()

,PerfScore,FromDiversityJobFair,Salary,Termd,PositionID,Zip,ManagerID,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,...,TeamIden,OrgIden,ConHour,CarOpp,PsySafe,Voice,Trust,OCB,*VoluntaryLeft,*MonthsSpent
count,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000,...,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000
mean,2.974277,0.093248,69020.684887,0.334405,16.845659,6555.482315,14.247588,4.110000,3.890675,1.218650,...,3.144695,2.739550,36.617363,2.488746,2.488746,3.080386,3.273312,2.913183,0.282958,105.836013
std,0.842029,0.291248,25156.636930,0.472542,6.223419,16908.396884,8.218770,0.789938,0.909241,2.349421,...,1.099103,0.807033,5.163603,1.037538,0.939648,1.105690,0.872091,0.843678,0.451162,26.314392
min,1.000000,0.000000,45046.000000,0.000000,1.000000,1013.000000,1.000000,1.120000,1.000000,0.000000,...,1.000000,1.000000,24.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,49.000000
25%,2.000000,0.000000,55501.500000,0.000000,18.000000,1901.500000,9.500000,3.690000,3.000000,0.000000,...,2.000000,2.000000,32.000000,2.000000,2.000000,2.000000,3.000000,2.000000,0.000000,92.500000
50%,3.000000,0.000000,62810.000000,0.000000,19.000000,2132.000000,14.000000,4.280000,4.000000,0.000000,...,3.000000,3.000000,40.000000,3.000000,2.000000,3.000000,3.000000,3.000000,0.000000,104.000000
75%,4.000000,0.000000,72036.000000,1.000000,20.000000,2355.000000,19.000000,4.700000,5.000000,0.000000,...,4.000000,3.000000,40.000000,3.000000,3.000000,4.000000,4.000000,3.000000,1.000000,122.000000
max,5.000000,1.000000,250000.000000,1.000000,30.000000,98052.000000,39.000000,5.000000,5.000000,8.000000,...,5.000000,5.000000,40.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1.000000,206.000000


In [10]:
# Engagement survey should be a float!!
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 43 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Employee.Name         311 non-null    object        
 1   PerfScore             311 non-null    int64         
 2   FromDiversityJobFair  311 non-null    int64         
 3   Salary                311 non-null    int64         
 4   Termd                 311 non-null    int64         
 5   PositionID            311 non-null    int64         
 6   Position              311 non-null    object        
 7   State                 311 non-null    object        
 8   Zip                   311 non-null    int64         
 9   DOB                   311 non-null    object        
 10  Sex                   311 non-null    object        
 11  MaritalDesc           311 non-null    object        
 12  CitizenDesc           311 non-null    object        
 13  HispanicLatino      

## Feature Selection

In [11]:
# Group by workers who 'left' the company
Left = df.groupby('Termd')
Left.mean()

,PerfScore,FromDiversityJobFair,Salary,PositionID,Zip,ManagerID,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,ManPos,...,TeamIden,OrgIden,ConHour,CarOpp,PsySafe,Voice,Trust,OCB,*VoluntaryLeft,*MonthsSpent
Termd,,,,,,,,,,,,,,,,,,,,,
0,3.135266,0.062802,70694.033816,16.198068,8218.772947,12.913043,4.119807,3.893720,1.463768,0.289855,...,3.367150,2.903382,36.657005,2.758454,2.705314,3.328502,3.541063,3.043478,0.000000,114.193237
1,2.653846,0.153846,65690.076923,18.134615,3244.894231,16.903846,4.090481,3.884615,0.730769,0.173077,...,2.701923,2.413462,36.538462,1.951923,2.057692,2.586538,2.740385,2.653846,0.846154,89.201923


In [12]:
# Group by the number of months spent in the company
Months_spent = df.groupby('*MonthsSpent')
Months_mean = Months_spent.mean()
Months_mean

,PerfScore,FromDiversityJobFair,Salary,Termd,PositionID,Zip,ManagerID,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,...,AssLead,TeamIden,OrgIden,ConHour,CarOpp,PsySafe,Voice,Trust,OCB,*VoluntaryLeft
*MonthsSpent,,,,,,,,,,,,,,,,,,,,,
49.0,3.500000,0.0,89885.500000,1.0,14.0,2014.5,9.000000,4.550000,4.0,2.000000,...,3.500000,3.000000,2.500000,40.000000,2.5,2.0,3.000000,3.000000,4.000000,0.5
50.0,3.000000,0.0,49920.000000,1.0,2.0,2170.0,1.000000,3.240000,3.0,4.000000,...,2.000000,1.000000,1.000000,40.000000,1.0,1.0,3.000000,2.000000,2.000000,0.0
51.0,2.000000,1.0,80512.000000,1.0,18.0,2478.0,2.000000,4.500000,3.0,0.000000,...,1.000000,4.000000,3.000000,32.000000,2.0,1.0,2.000000,4.000000,3.000000,1.0
55.0,3.000000,0.0,55578.000000,1.0,20.0,2138.0,20.000000,4.200000,5.0,0.000000,...,3.000000,2.000000,3.000000,40.000000,3.0,1.0,3.000000,2.000000,2.000000,1.0
56.0,3.500000,0.0,60234.500000,0.5,19.5,2291.5,21.000000,3.665000,4.5,1.500000,...,4.000000,2.500000,3.000000,40.000000,2.5,2.0,4.000000,3.000000,2.500000,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170.0,3.666667,0.0,111177.666667,0.0,18.0,1952.0,4.333333,4.143333,4.0,0.666667,...,3.666667,3.333333,2.666667,34.666667,3.0,2.0,2.666667,3.333333,3.666667,0.0
172.0,2.000000,1.0,63000.000000,0.0,1.0,1450.0,1.000000,4.500000,2.0,6.000000,...,3.000000,3.000000,3.000000,40.000000,2.0,2.0,4.000000,3.000000,4.000000,0.0
182.0,4.000000,0.0,64991.000000,0.0,19.0,1886.0,16.000000,4.840000,5.0,0.000000,...,4.000000,4.000000,3.000000,40.000000,4.0,3.0,4.000000,4.000000,4.000000,0.0


In [13]:
DeepnoteChart(Months_mean, """{"mark":{"type":"bar","tooltip":true},"width":"container","config":{"customFormatTypes":true},"height":"container","$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{"x":{"sort":null,"type":"nominal","field":"","scale":{"type":"linear"},"format":{"type":"default","decimals":null}},"y":{"sort":null,"type":"nominal","field":"","scale":{"type":"linear"},"format":{"type":"default","decimals":null}},"color":{"sort":null,"type":"nominal","field":"","scale":{"type":"linear"},"format":{"type":"default","decimals":null}}}}""")

In [14]:
left = df.loc[df['Termd']==True]
left_dummies = df_dummies.loc[df['Termd']==True]
working = df.loc[df['Termd']==False]
working_dummies = df_dummies.loc[df['Termd']==False]
voluntary_left = df.loc[df['*VoluntaryLeft']==True]
voluntary_left_dummies = df_dummies.loc[df['*VoluntaryLeft']==True]
print('Total number of employees that have left: ', len(left))
print('Of which, ', len(voluntary_left), ' have left voluntarily.')


Total number of employees that have left:  104
Of which,  88  have left voluntarily.


In [15]:
left

,Employee.Name,PerfScore,FromDiversityJobFair,Salary,Termd,PositionID,Position,State,Zip,DOB,...,OrgIden,ConHour,CarOpp,PsySafe,Voice,Trust,OCB,*VoluntaryLeft,*LastWorkingDate,*MonthsSpent
1,"Ait Sidi, Karthikeyan",4,0,104437,1,27,Sr. DBA,MA,2148,05/05/75,...,3,40,1,2,3,3,3,1,2020-06-16,63.0
2,"Akinkuolie, Sarah",2,0,64955,1,20,Production Technician II,MA,1810,09/19/88,...,2,40,1,2,2,2,1,1,2016-09-24,63.0
4,"Anderson, Carol",3,0,50825,1,19,Production Technician I,MA,2169,09/08/89,...,3,36,3,2,2,4,3,1,2020-09-06,110.0
10,"Baczenski, Rachael",3,1,54670,1,19,Production Technician I,MA,1902,01/12/74,...,3,40,2,1,2,4,3,1,2021-01-12,120.0
11,"Barbara, Thomas",2,1,47211,1,19,Production Technician I,MA,2062,02/21/74,...,3,40,1,2,1,3,2,1,2020-09-19,102.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,"Wilber, Barry",4,0,55140,1,19,Production Technician I,MA,2324,09/09/65,...,3,40,2,2,5,2,2,1,2019-09-07,100.0
302,"Wilkes, Annie",3,0,58062,1,19,Production Technician I,MA,1876,07/30/83,...,2,40,1,3,3,2,2,1,2016-05-14,64.0
303,"Williams, Jacquelyn",3,1,59728,1,19,Production Technician I,MA,2109,10/02/69,...,2,32,1,3,2,2,3,1,2019-06-27,90.0
304,"Winthrop, Jordan",3,0,70507,1,20,Production Technician II,MA,2045,11/07/58,...,2,40,2,1,3,4,3,1,2020-02-21,85.0


In [16]:
DeepnoteChart(left, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#9ecae9","tooltip":true},"encoding":{"x":{"sort":"ascending","type":"ordinal","field":"DateofTermination","scale":{"type":"linear"},"stack":"zero","timeUnit":"year"},"y":{"sort":null,"type":"quantitative","field":"Termd","scale":{"type":"linear"},"stack":"zero","format":{"type":"default","decimals":null},"aggregate":"sum","formatType":"numberFormatFromNumberType"},"color":{"datum":"Total Terminations"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#4c78a8","tooltip":true},"encoding":{"x":{"axis":{"title":"Termination Year "},"sort":"ascending","type":"ordinal","field":"DateofTermination","scale":{"type":"linear"},"stack":"zero","timeUnit":"year"},"y":{"axis":{"title":"Total Terminations"},"sort":null,"type":"quantitative","field":"*VoluntaryLeft","scale":{"type":"linear"},"stack":"zero","format":{"type":"default","decimals":null},"aggregate":"sum","formatType":"numberFormatFromNumberType"},"color":{"datum":"VoluntaryLeft"}}}]}],"title":"Total Terminations and Voluntary Terminations ","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

In [17]:
left

,Employee.Name,PerfScore,FromDiversityJobFair,Salary,Termd,PositionID,Position,State,Zip,DOB,...,OrgIden,ConHour,CarOpp,PsySafe,Voice,Trust,OCB,*VoluntaryLeft,*LastWorkingDate,*MonthsSpent
1,"Ait Sidi, Karthikeyan",4,0,104437,1,27,Sr. DBA,MA,2148,05/05/75,...,3,40,1,2,3,3,3,1,2020-06-16,63.0
2,"Akinkuolie, Sarah",2,0,64955,1,20,Production Technician II,MA,1810,09/19/88,...,2,40,1,2,2,2,1,1,2016-09-24,63.0
4,"Anderson, Carol",3,0,50825,1,19,Production Technician I,MA,2169,09/08/89,...,3,36,3,2,2,4,3,1,2020-09-06,110.0
10,"Baczenski, Rachael",3,1,54670,1,19,Production Technician I,MA,1902,01/12/74,...,3,40,2,1,2,4,3,1,2021-01-12,120.0
11,"Barbara, Thomas",2,1,47211,1,19,Production Technician I,MA,2062,02/21/74,...,3,40,1,2,1,3,2,1,2020-09-19,102.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,"Wilber, Barry",4,0,55140,1,19,Production Technician I,MA,2324,09/09/65,...,3,40,2,2,5,2,2,1,2019-09-07,100.0
302,"Wilkes, Annie",3,0,58062,1,19,Production Technician I,MA,1876,07/30/83,...,2,40,1,3,3,2,2,1,2016-05-14,64.0
303,"Williams, Jacquelyn",3,1,59728,1,19,Production Technician I,MA,2109,10/02/69,...,2,32,1,3,2,2,3,1,2019-06-27,90.0
304,"Winthrop, Jordan",3,0,70507,1,20,Production Technician II,MA,2045,11/07/58,...,2,40,2,1,3,4,3,1,2020-02-21,85.0


In [18]:
df_dummies.columns

Index(['Employee.Name', 'PerfScore', 'FromDiversityJobFair', 'Salary', 'Termd',
       'PositionID', 'Position', 'State', 'Zip', 'DOB', 'Sex', 'MaritalDesc',
       'CitizenDesc', 'HispanicLatino', 'DateofHire', 'DateofTermination',
       'TermReason', 'EmploymentStatus', 'ManagerName', 'RecruitmentSource',
       'EngagementSurvey', 'EmpSatisfaction', 'SpecialProjectsCount', 'ManPos',
       'TechLev', 'JobStr', 'AssColl', 'AssSelf', 'AssLead', 'TeamIden',
       'OrgIden', 'ConHour', 'CarOpp', 'PsySafe', 'Voice', 'Trust', 'OCB',
       '*VoluntaryLeft', '*LastWorkingDate', '*MonthsSpent',
       'Department_Admin Offices', 'Department_Executive Office',
       'Department_IT/IS', 'Department_Production       ', 'Department_Sales',
       'Department_Software Engineering',
       'RaceDesc_American Indian or Alaska Native', 'RaceDesc_Asian',
       'RaceDesc_Black or African American', 'RaceDesc_Hispanic',
       'RaceDesc_Two or more races', 'RaceDesc_White', 'ManagerID_1',
       '

In [19]:
# T-test
cols_d = ['PerfScore', 'FromDiversityJobFair', 'Salary', 'PositionID',
       'Zip','EngagementSurvey', 'EmpSatisfaction', 'SpecialProjectsCount', 'ManPos',
       'TechLev', 'JobStr', 'AssColl', 'AssSelf', 'AssLead', 'TeamIden',
       'OrgIden', 'ConHour', 'CarOpp', 'PsySafe', 'Voice', 'Trust', 'OCB',
       'Department_Admin Offices', 'Department_Executive Office',
       'Department_IT/IS', 'Department_Production       ', 'Department_Sales',
       'Department_Software Engineering',
       'RaceDesc_American Indian or Alaska Native', 'RaceDesc_Asian',
       'RaceDesc_Black or African American', 'RaceDesc_Hispanic',
       'RaceDesc_Two or more races', 'RaceDesc_White', 'ManagerID_1',
       'ManagerID_2', 'ManagerID_3', 'ManagerID_4', 'ManagerID_5',
       'ManagerID_6', 'ManagerID_7', 'ManagerID_9', 'ManagerID_10',
       'ManagerID_11', 'ManagerID_12', 'ManagerID_13', 'ManagerID_14',
       'ManagerID_15', 'ManagerID_16', 'ManagerID_17', 'ManagerID_18',
       'ManagerID_19', 'ManagerID_20', 'ManagerID_21', 'ManagerID_22',
       'ManagerID_30', 'ManagerID_39'
       ]

cols = ['PerfScore', 'FromDiversityJobFair', 'Salary', 'PositionID',
       'Zip','EngagementSurvey', 'EmpSatisfaction', 'SpecialProjectsCount', 'ManPos',
       'TechLev', 'JobStr', 'AssColl', 'AssSelf', 'AssLead', 'TeamIden',
       'OrgIden', 'ConHour', 'CarOpp', 'PsySafe', 'Voice', 'Trust', 'OCB'
       ]

print('P-Values:')
print('---------------------------------')
for i in cols_d:
    print(i,':  ',round(ttest_ind(left_dummies[i], working_dummies[i]).pvalue,4))

P-Values:
---------------------------------
PerfScore :   0.0
FromDiversityJobFair :   0.0091
Salary :   0.098
PositionID :   0.0094
Zip :   0.0141
EngagementSurvey :   0.758
EmpSatisfaction :   0.9338
SpecialProjectsCount :   0.0092
ManPos :   0.025
TechLev :   0.0
JobStr :   0.5787
AssColl :   0.0
AssSelf :   0.0
AssLead :   0.0
TeamIden :   0.0
OrgIden :   0.0
ConHour :   0.8489
CarOpp :   0.0
PsySafe :   0.0
Voice :   0.0
Trust :   0.0
OCB :   0.0001
Department_Admin Offices :   0.4707
Department_Executive Office :   0.4793
Department_IT/IS :   0.0279
Department_Production        :   0.0007
Department_Sales :   0.0314
Department_Software Engineering :   0.8349
RaceDesc_American Indian or Alaska Native :   0.2186
RaceDesc_Asian :   0.7739
RaceDesc_Black or African American :   0.5381
RaceDesc_Hispanic :   0.4793
RaceDesc_Two or more races :   0.6601
RaceDesc_White :   0.9092
ManagerID_1 :   0.9955
ManagerID_2 :   0.1972
ManagerID_3 :   0.4793
ManagerID_4 :   0.2222
ManagerID_5 :   0

### Logistic Regression

Function MODEL()  : 

In [20]:
def MODEL(X_train, y_train, X_test, y_test, model, params, scaler = None):
    """
    args:
        - X_train: a pandas DataFrame with features used to train model
        - y_train: a pandas DataFrame with target variables used to train model
        - X_test: a pandas DataFrame with features used to test model
        - y_test: a pandas DataFrame with target variables used to test model
        - model: model implemented
        - params: parameters used in Grid
        - scaler: scaler used in the dataset
        
    Returns grid. Trains and tests model. Prints results.
    """
    # Displaying Results
    def scores(grid,X_train,y_train,X_test,y_test):
        y_pred = grid.predict(X_test)
        print(f'Train score: {grid.score(X_train, y_train)}')
        print(f'Test score / Accuracy: {grid.score(X_test, y_test)}')
        f1_mic = f1_score(y_test, y_pred, average= "micro")
        f1_mac = f1_score(y_test, y_pred, average= "macro")
        prec_mic = precision_score(y_test, y_pred, average="micro")
        prec_mac = precision_score(y_test, y_pred, average="macro")
        rec_mic = recall_score(y_test, y_pred, average="micro")
        rec_mac = recall_score(y_test, y_pred, average="macro")
        print('                 Micro-Averaged          Macro-Averaged')
        print(f'F1             {f1_mic}      {f1_mac}  ')
        print(f'Precision      {prec_mic}      {prec_mac}')
        print(f'Recall         {rec_mic}      {rec_mac}')
        print('\n----------------------------------------------\n')
                
        
    # Pipelines / Gridsearch
    key = list(params.keys())[0]
    
    if scaler:
        print(f'Results using {scaler}:')
        pipe = make_pipeline(scaler, model)
        
        grid = GridSearchCV(pipe, params, cv=5)
        grid.fit(X_train, y_train)
              
        print("Best {}: ".format(key.split('__')[-1]), grid.best_params_[key])
        scores(grid,X_train, y_train, X_test,y_test)
    
    else:
        print(f'Results using no scaler:')
        grid = GridSearchCV(model, params, cv = 5)
        grid.fit(X_train, y_train)
        print("Best {}: ".format(key.split('__')[-1]), grid.best_params_[key])
        scores(grid, X_train, y_train, X_test, y_test)
        
                
                
    return grid

Function sig_features()  : 

In [21]:
def sig_features(grid: GridSearchCV, n: int = 15) -> np.array:
    """
     args:
        - grid: a sklearn GridSearchCV object trained on a training dataset
        - n: an integer that indicates the number of important features we are interested in; default is 15
        
    Returns a list, sig_features, of the features which correspond to the n heighest weights in the machine learning model.
        
    """
    model = grid.best_estimator_
    features = grid.feature_names_in_
    
    if type(model) == Pipeline:
        key = list(model.named_steps.keys())[1]
        coefs = abs(model.named_steps[key].coef_)
        coefs_reg = np.round(model.named_steps[key].coef_[0], 4)
    else:
        coefs = abs(model.coef_)
        coefs_reg = np.round(model.coef_[0], 4)
    
    starting = len(features) - n
    indices = np.argsort(coefs)[0][starting:]
    
    sig_features = []
    sig_coefs = []
    
    for index in indices:
        sig_features.append(features[index])
        sig_coefs.append(coefs_reg[index])
    
    indices = indices + 1
    
    return sig_features, indices, sig_coefs

In [22]:
# Target Variable: Voluntary Left (Dummies)
X_d = df_dummies[cols_d]
y_d = df_dummies['*VoluntaryLeft']

X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_d, y_d, test_size=0.2, random_state=100)

param_grid = {'logisticregression__C': np.logspace(-4,2,150)}

LR_vol = MODEL(X_train_d, y_train_d, X_test_d, y_test_d, LogisticRegression(penalty="l1",solver="liblinear",max_iter = 10e6, random_state = 0),param_grid, MinMaxScaler())

Results using MinMaxScaler():
Best C:  1.1671161911100683
Train score: 0.9354838709677419
Test score / Accuracy: 0.8571428571428571
                 Micro-Averaged          Macro-Averaged
F1             0.8571428571428571      0.8373027259684362  
Precision      0.8571428571428571      0.8418604651162791
Recall         0.8571428571428571      0.8333333333333333

----------------------------------------------



In [23]:
# Target Variable: Terminated (Dummies)
X_d = df_dummies[cols_d]
y_d = df_dummies['Termd']

X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_d, y_d, test_size=0.2, random_state=100)

param_grid = {'logisticregression__C': np.logspace(-4,2,150)}

LR_ter = MODEL(X_train_d, y_train_d, X_test_d, y_test_d, LogisticRegression(penalty="l1",solver="liblinear",max_iter = 10e6, random_state = 0),param_grid,StandardScaler())

Results using StandardScaler():
Best C:  0.34964245509531333
Train score: 0.9516129032258065
Test score / Accuracy: 0.9523809523809523
                 Micro-Averaged          Macro-Averaged
F1             0.9523809523809523      0.9505882352941175  
Precision      0.9523809523809523      0.9536842105263157
Recall         0.9523809523809523      0.9480249480249481

----------------------------------------------



Code below gives LR model without dummies (similar results, less informative)

In [24]:
# Target Variable: Voluntary Left
X = df[cols]    
y = df['*VoluntaryLeft']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100) 

param_grid = {'logisticregression__C': np.logspace(-4,2,150)}

LR_3 = MODEL(X_train, y_train, X_test, y_test, LogisticRegression(penalty="l1",solver="liblinear",max_iter = 10e6, random_state = 0),param_grid,MinMaxScaler())

Results using MinMaxScaler():
Best C:  6.795395920083764
Train score: 0.9274193548387096
Test score / Accuracy: 0.8412698412698413
                 Micro-Averaged          Macro-Averaged
F1             0.8412698412698413      0.8214285714285714  
Precision      0.8412698412698413      0.8214285714285714
Recall         0.8412698412698413      0.8214285714285714

----------------------------------------------



In [25]:
# Target Variable: Terminated 
X = df[cols]
y = df['Termd']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

param_grid = {'logisticregression__C': np.logspace(-4,2,150)}

LR = MODEL(X_train, y_train, X_test, y_test, LogisticRegression(penalty="l1",solver="liblinear",max_iter = 10e6, random_state = 0),param_grid,StandardScaler())

Results using StandardScaler():
Best C:  1.2805087496773324
Train score: 0.9475806451612904
Test score / Accuracy: 0.9523809523809523
                 Micro-Averaged          Macro-Averaged
F1             0.9523809523809523      0.9505882352941175  
Precision      0.9523809523809523      0.9536842105263157
Recall         0.9523809523809523      0.9480249480249481

----------------------------------------------



In [26]:
features, numbers, coefs = sig_features(LR_vol, 15)
imp_feats_vol_df= pd.DataFrame(list(zip(features, coefs)),columns=['Feature','Coefficients'] )
imp_feats_vol_df

,Feature,Coefficients
0,FromDiversityJobFair,0.9359
1,ManagerID_21,-0.9440
2,Voice,-1.0417
3,TechLev,-1.1891
4,AssColl,-1.2123
5,ManagerID_11,1.2205
6,PerfScore,-1.3070
7,OrgIden,-1.7345
8,OCB,-2.0572
9,Trust,-2.1243


In [27]:
features, numbers, coefs = sig_features(LR_ter, 15)
imp_feats_ter_df = pd.DataFrame(list(zip(features, coefs)),columns=['Feature','Coefficients'] )
imp_feats_ter_df_filtered = imp_feats_ter_df.drop([0,2,4,7,8,10,14])
imp_feats_ter_df_filtered

,Feature,Coefficients
1,TeamIden,-0.2139
3,Voice,-0.2462
5,OrgIden,-0.4179
6,OCB,-0.4284
9,Trust,-0.6079
11,PsySafe,-1.2595
12,EmpSatisfaction,1.4706
13,EngagementSurvey,1.5025


### Important Features -> to predict if a worker leaves/stays

In [28]:
DeepnoteChart(imp_feats_vol_df, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#4c78a8","tooltip":true},"encoding":{"x":{"sort":null,"type":"quantitative","field":"Coefficients","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"y":{"sort":{"order":"ascending","encoding":"x"},"type":"nominal","field":"Feature","scale":{"type":"linear"}}}}]}],"title":"Important Features -> Leaving voluntarily","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

In [29]:
DeepnoteChart(imp_feats_ter_df, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#4c78a8","tooltip":true},"encoding":{"x":{"sort":null,"type":"quantitative","field":"Coefficients","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"y":{"sort":{"order":"ascending","encoding":"x"},"type":"nominal","field":"Feature","scale":{"type":"linear"}}}}]}],"title":"Important Features -> Terminating","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

In [30]:
DeepnoteChart(imp_feats_ter_df_filtered, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#4c78a8","tooltip":true},"encoding":{"x":{"sort":null,"type":"quantitative","field":"Coefficients","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"y":{"sort":{"order":"ascending","encoding":"x"},"type":"nominal","field":"Feature","scale":{"type":"linear"}}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

### Features linked to working Environment

Features that have a strong impact in determining whether an employee stays/leaves in the company: <- these are the areas we should focus on improving (how? check papers)

- PsySafe (Psychological Safety)

- Trust

- Voice

- OCB (Colleague’s reported organisational citizen behaviour)

- EmpSatisfaction

- OrgIden (Identification with the organisation)

- TeamIden (Identification with team)

!!! In addition to these, we notice that the variable Career Opportunities has a strong influence on the decision of a worker staying/leaving (Point 3 of analysis in the report ??)

### Updated DataFrame and Talent Define

In [31]:
pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 KB 3.7 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [32]:
data = pd.read_excel('data_all.xlsx')
data_active = pd.read_excel("data_active.xlsx")
data_cause_leaving = pd.read_excel("data_cause_leaving.xlsx")
data_v = pd.read_excel('data_voluntarily_leaving.xlsx')

In [33]:
df["*DateDiff"] = data['DateDiff']

In [34]:
weights = [0.2, 0.2, 0.2, 0.2,0.2]
df["*TalentScore"] = df[['PerfScore', 'AssColl', 'AssSelf', 'AssLead', 'TechLev',]].apply(lambda x: pd.Series.dot(x, weights), axis=1).round(2)

In [35]:
percentile_85 = df['*TalentScore'].quantile(q=0.85)

print(percentile_85)

4.0


In [36]:
def talent_score_to_binary(score):
    if score >= percentile_85:
        return 1
    else:
        return 0


df['*Talent'] = df['*TalentScore'].apply(talent_score_to_binary)

In [37]:
df['*Talent'].value_counts()

0    243
1     68
Name: *Talent, dtype: int64

In [38]:
df_environment = df.drop(columns=['CarOpp', 'SpecialProjectsCount','ManPos','TechLev','JobStr',
'AssColl','AssLead','ConHour','RecruitmentSource','MaritalDesc','DOB',
'Zip','State','FromDiversityJobFair','PerfScore', 'AssSelf'])
df_environment = df_environment.drop(150) # drop CFO
df_environment

,Employee.Name,Salary,Termd,PositionID,Position,Sex,CitizenDesc,HispanicLatino,RaceDesc,DateofHire,...,PsySafe,Voice,Trust,OCB,*VoluntaryLeft,*LastWorkingDate,*MonthsSpent,*DateDiff,*TalentScore,*Talent
0,"Adinolfi, Wilson K",62506,0,19,Production Technician I,M,US Citizen,No,White,2011-07-05,...,4,5,5,4,0,2023-02-27,140.0,8,3.4,0
1,"Ait Sidi, Karthikeyan",104437,1,27,Sr. DBA,M,US Citizen,No,White,2015-03-30,...,2,3,3,3,1,2020-06-16,63.0,9,3.6,0
2,"Akinkuolie, Sarah",64955,1,20,Production Technician II,F,US Citizen,No,White,2011-07-05,...,2,2,2,1,1,2016-09-24,63.0,11,2.4,0
3,"Alagbe,Trina",64991,0,19,Production Technician I,F,US Citizen,No,White,2008-01-07,...,3,4,4,4,0,2023-02-27,182.0,6,4.0,1
4,"Anderson, Carol",50825,1,19,Production Technician I,F,US Citizen,No,White,2011-07-11,...,2,2,4,3,1,2020-09-06,110.0,9,3.6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,"Woodson, Jason",65893,0,20,Production Technician II,M,US Citizen,No,White,2014-07-07,...,2,3,3,3,0,2023-02-27,104.0,11,3.6,0
307,"Ybarra, Catherine",48513,1,19,Production Technician I,F,US Citizen,No,Asian,2008-09-02,...,1,1,3,1,1,2019-09-29,133.0,9,3.0,0
308,"Zamora, Jennifer",220450,0,6,CIO,F,US Citizen,No,White,2010-04-10,...,3,5,4,3,0,2023-02-27,155.0,8,3.8,0
309,"Zhou, Julia",89292,0,9,Data Analyst,F,US Citizen,No,White,2015-03-30,...,1,2,3,3,0,2023-02-27,95.0,7,4.2,1


## Analysis

In [39]:
DeepnoteChart(voluntary_left, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#e45756","tooltip":true},"encoding":{"x":{"sort":null,"type":"quantitative","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"aggregate":"count","formatType":"numberFormatFromNumberType"},"y":{"sort":{"order":"descending","encoding":"x"},"type":"nominal","field":"TermReason","scale":{"type":"linear"}}}}]}],"title":"Termination Reasons (Voluntary)","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

→ Out of the 88 people that have left voluntarily: 

Lack of Career Opportunities: 20 another position (23%) / 9 career change (10%) / 5 return to school (6%) => 39 % 

Unhappy: 14 unhappy (16%) / 1 attendance (1%) / 1 performance (1%) / 3 maternity no return (3%)

Research shows that unhappiness at work can lead to lack of attendance, low performance, no-call, no-show

In [40]:
DeepnoteChart(left, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#4c78a8","tooltip":true},"encoding":{"x":{"sort":null,"type":"quantitative","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"aggregate":"count","formatType":"numberFormatFromNumberType"},"y":{"sort":{"order":"descending","encoding":"x"},"type":"nominal","field":"TermReason","scale":{"type":"linear"}}}}]}],"title":"Termination Reasons (Voluntary & Involuntary)","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

→  Out of the 104 people that no longer work in the company:

Lack of career opportunities: 9 career change (9%) / 5 return to school (5%) / 20 another position (19%) => 33%

Unhappy: 7 attendance (7%) / 4 performance (4%) / 4 no-call no-show (4%) / 3 maternity no return (3%) => 18%

### Hypothesis 1: There is a better working environment in particular departments

In [41]:
Deps = df_environment[['Department','Trust','OCB', 'PsySafe','Voice','EmpSatisfaction','OrgIden','TeamIden','Employee.Name','Termd','*VoluntaryLeft']].groupby('Department')
Deps_mean = Deps.mean().round(2)
Deps_mean['Average']=Deps_mean.iloc[: , :-1].mean(axis=1).round(2)
Deps_mean= Deps_mean.sort_values(by=['*VoluntaryLeft'])
Deps_mean.reset_index(inplace=True)
Deps_mean

,Department,Trust,OCB,PsySafe,Voice,EmpSatisfaction,OrgIden,TeamIden,Termd,*VoluntaryLeft,Average
0,Sales,3.35,2.87,2.65,3.32,4.03,2.81,3.39,0.16,0.10,2.82
1,Admin Offices,3.22,2.78,2.22,2.67,3.56,3.00,3.11,0.22,0.11,2.60
2,IT/IS,3.32,2.96,2.54,3.32,3.96,2.82,3.08,0.20,0.12,2.78
3,Software Engineering,3.18,2.91,2.36,3.09,4.09,2.64,3.45,0.36,0.27,2.76
4,Production,3.26,2.91,2.48,3.01,3.86,2.70,3.11,0.40,0.36,2.72


In [42]:
DeepnoteChart(Deps_mean, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#4c78a8","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"Department","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"PsySafe","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"PsySafe"},"xOffset":{"datum":"PsySafe"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"Department","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"Trust","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"Trust"},"xOffset":{"datum":"Trust"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#e45756","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"Department","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"Voice","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"Voice"},"xOffset":{"datum":"Voice"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#72b7b2","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"Department","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"EmpSatisfaction","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"EmpSatisfaction"},"xOffset":{"datum":"EmpSatisfaction"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#54a24b","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"Department","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"OrgIden","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"OrgIden"},"xOffset":{"datum":"OrgIden"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#eeca3b","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"Department","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"TeamIden","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"TeamIden"},"xOffset":{"datum":"TeamIden"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#b279a2","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"Department","scale":{"type":"linear"}},"y":{"axis":{"title":"Ratings"},"sort":null,"type":"quantitative","field":"OCB","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"OCB"},"xOffset":{"datum":"OCB"}}}]},{"layer":[{"mark":{"clip":true,"type":"trail","color":"#000","tooltip":true},"encoding":{"x":{"sort":{"order":"ascending","encoding":"y"},"type":"nominal","field":"Department","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"quantitative","field":"Average","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"}}},{"mark":{"dx":0,"dy":-1,"fill":"black","type":"text","align":"center","baseline":"bottom"},"encoding":{"x":{"sort":{"order":"ascending","encoding":"y"},"type":"nominal","field":"Department","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"quantitative","field":"Average","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"text":{"sort":null,"type":"quantitative","field":"Average"}}},{"mark":{"size":100,"type":"point","opacity":0,"tooltip":true},"encoding":{"x":{"sort":{"order":"ascending","encoding":"y"},"type":"nominal","field":"Department","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"quantitative","field":"Average","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

In [43]:
DeepnoteChart(Deps_mean, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#bab0ac","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"Department","scale":{"type":"linear"}},"y":{"axis":{"title":"Voluntary Terminations","format":".0%"},"sort":null,"type":"quantitative","field":"*VoluntaryLeft","scale":{"type":"linear","domainMax":0.7},"format":{"type":"percent","decimals":null},"aggregate":"sum","formatType":"numberFormatFromNumberType"}}}]},{"layer":[{"mark":{"clip":true,"type":"trail","color":"#54a24b","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"Department","scale":{"type":"linear","zero":false}},"y":{"axis":{"title":null},"sort":null,"type":"quantitative","field":"Average","scale":{"type":"linear","zero":false,"domainMax":2.9,"domainMin":2.5},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"Average"}}},{"mark":{"size":100,"type":"point","opacity":0,"tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"Department","scale":{"type":"linear","zero":false}},"y":{"axis":{"title":null},"sort":null,"type":"quantitative","field":"Average","scale":{"type":"linear","zero":false,"domainMax":2.9,"domainMin":2.5},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"Average"}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","resolve":{"scale":{"y":"independent"}},"encoding":{}}""")

-> Departments with a low Psychological Safety / Voice / Trust are the ones with the highest percentage of terminations. (With the exception of the 

The Sales department has the highest indicators and is the department

In [44]:
Term = df_environment[['Department','Trust','OCB', 'PsySafe','Voice','EmpSatisfaction','OrgIden','TeamIden','Employee.Name','Termd']].groupby('Termd')
Term_mean = Term.mean().round(2)
Term_mean['Average']=Term_mean.iloc[: , :-1].mean(axis=1).round(2)
Term_mean.reset_index(inplace=True)
Term_mean = Term_mean.T.drop(['Termd'])
Term_mean.reset_index(inplace=True)
Term_mean.columns = ['Indicator', 'NO', 'YES']
Term_mean




,Indicator,NO,YES
0,Trust,3.54,2.74
1,OCB,3.04,2.65
2,PsySafe,2.71,2.06
3,Voice,3.34,2.59
4,EmpSatisfaction,3.90,3.88
5,OrgIden,2.90,2.41
6,TeamIden,3.36,2.70
7,Average,3.24,2.72


In [45]:
DeepnoteChart(Term_mean, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#e45756","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"Indicator","scale":{"type":"linear"}},"y":{"axis":{"title":"Scores"},"sort":null,"type":"quantitative","field":"YES","scale":{"type":"linear","domainMin":1.6},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"YES"},"xOffset":{"datum":"YES"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"Indicator","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"NO","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"NO"},"xOffset":{"datum":"NO"}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

In [46]:
# T-Test
# Null Hypothesis: Work environment indicators are the same for all departments (Same work environment)
# Alternative Hypothesis: Some departments have a worse working environment


print('-> Production & Software Engineering')
for i in ['PsySafe','Trust', 'Voice','EmpSatisfaction','OrgIden','TeamIden','OCB']:
    print(i,': ',round(ttest_ind(df_environment[i][df_environment['Department'] == 'Production       '], df_environment[i][df_environment['Department'] == 'Software Engineering']).pvalue,4))

print('')
print('-> Production & Sales')
for i in ['PsySafe','Trust', 'Voice','EmpSatisfaction','OrgIden','TeamIden','OCB']:
    print(i,': ',round(ttest_ind(df_environment[i][df_environment['Department'] == 'Production       '], df_environment[i][df_environment['Department'] == 'Sales']).pvalue,4))

print('')
print('-> Production & IT')
for i in ['PsySafe','Trust', 'Voice','EmpSatisfaction','OrgIden','TeamIden','OCB']:
    print(i,': ',round(ttest_ind(df_environment[i][df_environment['Department'] == 'Production       '], df_environment[i][df_environment['Department'] == 'IT/IS']).pvalue,4))

print('')
print('-> Production & Admin Offices')
for i in ['PsySafe','Trust', 'Voice','EmpSatisfaction','OrgIden','TeamIden','OCB']:
    print(i,': ',round(ttest_ind(df_environment[i][df_environment['Department'] == 'Production       '], df_environment[i][df_environment['Department'] == 'Sales']).pvalue,4))

print('')
print('-> Software & Sales')
for i in ['PsySafe','Trust', 'Voice','EmpSatisfaction','OrgIden','TeamIden','OCB']:
    print(i,': ',round(ttest_ind(df_environment[i][df_environment['Department'] == 'Software Engineering'], df_environment[i][df_environment['Department'] == 'Sales']).pvalue,4))

print('')
print('-> Software & IT')
for i in ['PsySafe','Trust', 'Voice','EmpSatisfaction','OrgIden','TeamIden','OCB']:
    print(i,': ',round(ttest_ind(df_environment[i][df_environment['Department'] == 'Software Engineering'], df_environment[i][df_environment['Department'] == 'IT/IS']).pvalue,4))

print('')
print('-> Software & Admin Offices')
for i in ['PsySafe','Trust', 'Voice','EmpSatisfaction','OrgIden','TeamIden','OCB']:
    print(i,': ',round(ttest_ind(df_environment[i][df_environment['Department'] == 'Software Engineering'], df_environment[i][df_environment['Department'] == 'Admin Offices']).pvalue,4))

print('')
print('-> Sales & IT')
for i in ['PsySafe','Trust', 'Voice','EmpSatisfaction','OrgIden','TeamIden','OCB']:
    print(i,': ',round(ttest_ind(df_environment[i][df_environment['Department'] == 'Sales'], df_environment[i][df_environment['Department'] == 'IT/IS']).pvalue,4))

print('')
print('-> Sales & Admin Offices')
for i in ['PsySafe','Trust', 'Voice','EmpSatisfaction','OrgIden','TeamIden','OCB']:
    print(i,': ',round(ttest_ind(df_environment[i][df_environment['Department'] == 'Sales'], df_environment[i][df_environment['Department'] == 'Admin Offices']).pvalue,4))

print('')
print('-> IT & Admin Offices')
for i in ['PsySafe','Trust', 'Voice','EmpSatisfaction','OrgIden','TeamIden','OCB']:
    print(i,': ',round(ttest_ind(df_environment[i][df_environment['Department'] == 'IT/IS'], df_environment[i][df_environment['Department'] == 'Admin Offices']).pvalue,4))



-> Production & Software Engineering
PsySafe :  0.6822
Trust :  0.7854
Voice :  0.8235
EmpSatisfaction :  0.4094
OrgIden :  0.7874
TeamIden :  0.3171
OCB :  1.0

-> Production & Sales
PsySafe :  0.352
Trust :  0.5785
Voice :  0.1494
EmpSatisfaction :  0.3357
OrgIden :  0.5094
TeamIden :  0.1942
OCB :  0.8214

-> Production & IT
PsySafe :  0.6762
Trust :  0.6599
Voice :  0.0868
EmpSatisfaction :  0.4808
OrgIden :  0.3673
TeamIden :  0.8846
OCB :  0.6973

-> Production & Admin Offices
PsySafe :  0.352
Trust :  0.5785
Voice :  0.1494
EmpSatisfaction :  0.3357
OrgIden :  0.5094
TeamIden :  0.1942
OCB :  0.8214

-> Software & Sales
PsySafe :  0.4251
Trust :  0.5342
Voice :  0.4761
EmpSatisfaction :  0.874
OrgIden :  0.5112
TeamIden :  0.8523
OCB :  0.9123

-> Software & IT
PsySafe :  0.5983
Trust :  0.5895
Voice :  0.5295
EmpSatisfaction :  0.6537
OrgIden :  0.4943
TeamIden :  0.2462
OCB :  0.8383

-> Software & Admin Offices
PsySafe :  0.7057
Trust :  0.9175
Voice :  0.241
EmpSatisfaction 

In [47]:
# T-Test
# Null Hypothesis: Work environment indicators are the same for all departments (Same work environment)
# Alternative Hypothesis: Some departments have a worse working environment


print('-> Production & Software Engineering')
for i in ['PsySafe','Trust', 'Voice','EmpSatisfaction','OrgIden','TeamIden','OCB']:
    print(i,': ',round(ttest_ind(df_environment[i][df_environment['Department'] == 'Production       '], df_environment[i][df_environment['Department'] == 'Software Engineering']).pvalue,4))

print('')
print('-> Production & Sales')
for i in ['PsySafe','Trust', 'Voice','EmpSatisfaction','OrgIden','TeamIden','OCB']:
    print(i,': ',round(ttest_ind(df_environment[i][df_environment['Department'] == 'Production       '], df_environment[i][df_environment['Department'] == 'Sales']).pvalue,4))

print('')
print('-> Production & IT')
for i in ['PsySafe','Trust', 'Voice','EmpSatisfaction','OrgIden','TeamIden','OCB']:
    print(i,': ',round(ttest_ind(df_environment[i][df_environment['Department'] == 'Production       '], df_environment[i][df_environment['Department'] == 'IT/IS']).pvalue,4))

print('')
print('-> Production & Admin Offices')
for i in ['PsySafe','Trust', 'Voice','EmpSatisfaction','OrgIden','TeamIden','OCB']:
    print(i,': ',round(ttest_ind(df_environment[i][df_environment['Department'] == 'Production       '], df_environment[i][df_environment['Department'] == 'Sales']).pvalue,4))

print('')
print('-> Software & Sales')
for i in ['PsySafe','Trust', 'Voice','EmpSatisfaction','OrgIden','TeamIden','OCB']:
    print(i,': ',round(ttest_ind(df_environment[i][df_environment['Department'] == 'Software Engineering'], df_environment[i][df_environment['Department'] == 'Sales']).pvalue,4))

print('')
print('-> Software & IT')
for i in ['PsySafe','Trust', 'Voice','EmpSatisfaction','OrgIden','TeamIden','OCB']:
    print(i,': ',round(ttest_ind(df_environment[i][df_environment['Department'] == 'Software Engineering'], df_environment[i][df_environment['Department'] == 'IT/IS']).pvalue,4))

print('')
print('-> Software & Admin Offices')
for i in ['PsySafe','Trust', 'Voice','EmpSatisfaction','OrgIden','TeamIden','OCB']:
    print(i,': ',round(ttest_ind(df_environment[i][df_environment['Department'] == 'Software Engineering'], df_environment[i][df_environment['Department'] == 'Admin Offices']).pvalue,4))

print('')
print('-> Sales & IT')
for i in ['PsySafe','Trust', 'Voice','EmpSatisfaction','OrgIden','TeamIden','OCB']:
    print(i,': ',round(ttest_ind(df_environment[i][df_environment['Department'] == 'Sales'], df_environment[i][df_environment['Department'] == 'IT/IS']).pvalue,4))

print('')
print('-> Sales & Admin Offices')
for i in ['PsySafe','Trust', 'Voice','EmpSatisfaction','OrgIden','TeamIden','OCB']:
    print(i,': ',round(ttest_ind(df_environment[i][df_environment['Department'] == 'Sales'], df_environment[i][df_environment['Department'] == 'Admin Offices']).pvalue,4))

print('')
print('-> IT & Admin Offices')
for i in ['PsySafe','Trust', 'Voice','EmpSatisfaction','OrgIden','TeamIden','OCB']:
    print(i,': ',round(ttest_ind(df_environment[i][df_environment['Department'] == 'IT/IS'], df_environment[i][df_environment['Department'] == 'Admin Offices']).pvalue,4))



-> Production & Software Engineering
PsySafe :  0.6822
Trust :  0.7854
Voice :  0.8235
EmpSatisfaction :  0.4094
OrgIden :  0.7874
TeamIden :  0.3171
OCB :  1.0

-> Production & Sales
PsySafe :  0.352
Trust :  0.5785
Voice :  0.1494
EmpSatisfaction :  0.3357
OrgIden :  0.5094
TeamIden :  0.1942
OCB :  0.8214

-> Production & IT
PsySafe :  0.6762
Trust :  0.6599
Voice :  0.0868
EmpSatisfaction :  0.4808
OrgIden :  0.3673
TeamIden :  0.8846
OCB :  0.6973

-> Production & Admin Offices
PsySafe :  0.352
Trust :  0.5785
Voice :  0.1494
EmpSatisfaction :  0.3357
OrgIden :  0.5094
TeamIden :  0.1942
OCB :  0.8214

-> Software & Sales
PsySafe :  0.4251
Trust :  0.5342
Voice :  0.4761
EmpSatisfaction :  0.874
OrgIden :  0.5112
TeamIden :  0.8523
OCB :  0.9123

-> Software & IT
PsySafe :  0.5983
Trust :  0.5895
Voice :  0.5295
EmpSatisfaction :  0.6537
OrgIden :  0.4943
TeamIden :  0.2462
OCB :  0.8383

-> Software & Admin Offices
PsySafe :  0.7057
Trust :  0.9175
Voice :  0.241
EmpSatisfaction 

In [48]:
production = df[df['Department']=='Production       ']
production   

,Employee.Name,PerfScore,FromDiversityJobFair,Salary,Termd,PositionID,Position,State,Zip,DOB,...,PsySafe,Voice,Trust,OCB,*VoluntaryLeft,*LastWorkingDate,*MonthsSpent,*DateDiff,*TalentScore,*Talent
0,"Adinolfi, Wilson K",3,0,62506,0,19,Production Technician I,MA,1960,07/10/83,...,4,5,5,4,0,2023-02-27,140.0,8,3.4,0
2,"Akinkuolie, Sarah",2,0,64955,1,20,Production Technician II,MA,1810,09/19/88,...,2,2,2,1,1,2016-09-24,63.0,11,2.4,0
3,"Alagbe,Trina",4,0,64991,0,19,Production Technician I,MA,1886,09/27/88,...,3,4,4,4,0,2023-02-27,182.0,6,4.0,1
4,"Anderson, Carol",3,0,50825,1,19,Production Technician I,MA,2169,09/08/89,...,2,2,4,3,1,2020-09-06,110.0,9,3.6,0
5,"Anderson, Linda",3,0,57568,0,19,Production Technician I,MA,1844,05/22/77,...,2,1,4,4,0,2023-02-27,134.0,8,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,"Winthrop, Jordan",3,0,70507,1,20,Production Technician II,MA,2045,11/07/58,...,1,3,4,3,1,2020-02-21,85.0,5,3.2,0
305,"Wolk, Hang T",3,0,60446,0,20,Production Technician II,MA,2302,04/20/85,...,2,3,4,2,0,2023-02-27,101.0,11,3.4,0
306,"Woodson, Jason",3,0,65893,0,20,Production Technician II,MA,1810,05/11/85,...,2,3,3,3,0,2023-02-27,104.0,11,3.6,0
307,"Ybarra, Catherine",3,0,48513,1,19,Production Technician I,MA,2458,05/04/82,...,1,1,3,1,1,2019-09-29,133.0,9,3.0,0


### Hypothesis 2: (A) particular manager(s) create(s) a better working environment than others

In [49]:
Managers = df_environment[['ManagerName','Trust','OCB', 'Voice','EmpSatisfaction','PsySafe','OrgIden','TeamIden','Employee.Name','*VoluntaryLeft','Termd']].groupby('ManagerName')
Managers_mean = Managers.mean().round(2)
Managers_mean['Average']=Managers_mean.iloc[: , :-1].mean(axis=1).round(2)
Managers_mean.reset_index(inplace=True)
Managers_mean_1 = Managers_mean.sort_values(by=['Average']).reset_index()
Managers_mean = Managers_mean_1.drop([0,3,6,7,8,9,10,11,12,13,14,15,16], axis=0)
Managers_mean

,index,ManagerName,Trust,OCB,Voice,EmpSatisfaction,PsySafe,OrgIden,TeamIden,*VoluntaryLeft,Termd,Average
1,1,Amy Dunn,3.10,2.62,2.71,3.81,2.38,2.71,2.57,0.62,0.62,2.57
2,15,Kissy Sullivan,2.95,2.64,2.77,3.91,2.32,2.82,2.95,0.45,0.55,2.60
4,3,Brandon R. LeBlanc,3.29,2.86,2.71,3.57,2.00,3.00,3.43,0.14,0.29,2.62
5,4,Brannon Miller,3.41,2.82,3.14,3.41,2.45,2.50,3.00,0.27,0.27,2.62
17,18,Peter Monroe,3.36,2.93,3.86,3.93,2.71,2.79,3.21,0.07,0.07,2.86
18,9,Eric Dougall,3.75,3.50,3.50,3.50,2.75,2.50,4.00,0.00,0.00,2.94
19,11,Jennifer Zamora,3.71,2.71,3.57,4.29,3.29,2.86,3.00,0.14,0.14,2.95
20,7,Debra Houlihan,4.00,3.67,3.33,4.33,2.67,2.33,4.33,0.33,0.33,3.12


In [50]:
DeepnoteChart(Managers_mean, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#ff9da6","tooltip":true},"encoding":{"x":{"axis":{"title":"Manager Name"},"sort":null,"type":"nominal","field":"ManagerName","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"PsySafe","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"PsySafe"},"xOffset":{"datum":"PsySafe"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#4c78a8","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"ManagerName","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"Trust","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"Trust"},"xOffset":{"datum":"Trust"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#e45756","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"ManagerName","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"Voice","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"Voice"},"xOffset":{"datum":"Voice"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#72b7b2","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"ManagerName","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"EmpSatisfaction","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"EmpSatisfaction"},"xOffset":{"datum":"EmpSatisfaction"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#eeca3b","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"ManagerName","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"OrgIden","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"OrgIden"},"xOffset":{"datum":"OrgIden"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#54a24b","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"ManagerName","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"TeamIden","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"TeamIden"},"xOffset":{"datum":"TeamIden"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"ManagerName","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"OCB","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"OCB"},"xOffset":{"datum":"OCB"}}}]},{"layer":[{"mark":{"clip":true,"type":"trail","color":"#000","tooltip":true},"encoding":{"x":{"sort":{"order":"ascending","encoding":"y"},"type":"nominal","field":"ManagerName","scale":{"type":"linear","zero":false}},"y":{"axis":{"title":"Ratings"},"sort":null,"type":"quantitative","field":"Average","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"}}},{"mark":{"dx":0,"dy":-1,"fill":"black","type":"text","align":"center","baseline":"bottom"},"encoding":{"x":{"sort":{"order":"ascending","encoding":"y"},"type":"nominal","field":"ManagerName","scale":{"type":"linear","zero":false}},"y":{"axis":{"title":"Ratings"},"sort":null,"type":"quantitative","field":"Average","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"text":{"sort":null,"type":"quantitative","field":"Average"}}},{"mark":{"size":100,"type":"point","opacity":0,"tooltip":true},"encoding":{"x":{"sort":{"order":"ascending","encoding":"y"},"type":"nominal","field":"ManagerName","scale":{"type":"linear","zero":false}},"y":{"axis":{"title":"Ratings"},"sort":null,"type":"quantitative","field":"Average","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

In [51]:
DeepnoteChart(Managers_mean_1, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"circle","color":"#bab0ac","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"ManagerName","scale":{"type":"linear","zero":false}},"y":{"axis":{"format":".0%"},"sort":null,"type":"quantitative","field":"Termd","scale":{"type":"linear","zero":false,"domainMax":0.8},"format":{"type":"percent","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"Termd"}}}]},{"layer":[{"mark":{"clip":true,"type":"trail","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"ManagerName","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"quantitative","field":"Average","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"Average"}}},{"mark":{"size":100,"type":"point","opacity":0,"tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"ManagerName","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"quantitative","field":"Average","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"Average"}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","resolve":{"scale":{"y":"independent"}},"encoding":{}}""")

There does no seem to =>

### Hypothesis 3: There is lack of diversity => Worse working environment

- For Race :

In [52]:
Races = df_environment[['RaceDesc','Trust','OCB', 'Voice','EmpSatisfaction','PsySafe','OrgIden','TeamIden','Employee.Name','*VoluntaryLeft','Termd']].groupby("RaceDesc")
Races_mean = Races.mean().round(2)
Races_mean['Average']=Races_mean.iloc[: , :-1].mean(axis=1).round(2)
Races_mean
Races_mean.reset_index(inplace=True)
Races_mean

,RaceDesc,Trust,OCB,Voice,EmpSatisfaction,PsySafe,OrgIden,TeamIden,*VoluntaryLeft,Termd,Average
0,American Indian or Alaska Native,3.67,3.00,3.67,4.67,3.00,3.00,3.67,0.00,0.00,3.08
1,Asian,3.24,2.93,2.83,3.72,2.48,2.69,3.21,0.31,0.31,2.68
2,Black or African American,3.09,2.99,3.28,3.94,2.40,2.82,3.16,0.29,0.36,2.75
3,Hispanic,3.00,4.00,3.00,3.00,4.00,2.00,3.00,0.00,0.00,2.75
4,Two or more races,3.09,2.64,3.18,3.82,2.82,2.73,3.00,0.18,0.27,2.68
5,White,3.37,2.88,3.03,3.90,2.50,2.71,3.12,0.29,0.34,2.72


In [53]:
DeepnoteChart(df_environment, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#e45756","tooltip":true},"encoding":{"x":{"sort":null,"type":"quantitative","field":"RaceDesc","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"aggregate":"count","formatType":"numberFormatFromNumberType"},"y":{"sort":null,"type":"nominal","field":"RaceDesc","scale":{"type":"linear"}}}}]}],"title":"Number of people per Race","config":{"legend":{"orient":"top"}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

In [54]:
DeepnoteChart(Races_mean, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#4c78a8","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"RaceDesc","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"PsySafe","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"PsySafe"},"xOffset":{"datum":"PsySafe"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"RaceDesc","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"OrgIden","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"OrgIden"},"xOffset":{"datum":"OrgIden"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#e45756","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"RaceDesc","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"TeamIden","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"TeamIden"},"xOffset":{"datum":"TeamIden"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#72b7b2","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"RaceDesc","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"EmpSatisfaction","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"EmpSatisfaction"},"xOffset":{"datum":"EmpSatisfaction"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#54a24b","tooltip":true},"encoding":{"x":{"sort":null,"scale":{"type":"linear"}},"y":{"sort":null,"scale":{"type":"linear"}}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#eeca3b","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"RaceDesc","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"Voice","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"Voice"},"xOffset":{"datum":"Voice"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#b279a2","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"RaceDesc","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"OCB","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"OCB"},"xOffset":{"datum":"OCB"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#ff9da6","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"RaceDesc","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"Trust","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"Trust"},"xOffset":{"datum":"Trust"}}}]},{"layer":[{"mark":{"clip":true,"type":"trail","color":"#000","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"RaceDesc","scale":{"type":"linear","zero":false}},"y":{"axis":{"title":"Ratings"},"sort":null,"type":"quantitative","field":"Average","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"}}},{"mark":{"dx":0,"dy":-1,"fill":"black","type":"text","align":"center","baseline":"bottom"},"encoding":{"x":{"sort":null,"type":"nominal","field":"RaceDesc","scale":{"type":"linear","zero":false}},"y":{"axis":{"title":"Ratings"},"sort":null,"type":"quantitative","field":"Average","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"text":{"sort":null,"type":"quantitative","field":"Average"}}},{"mark":{"size":100,"type":"point","opacity":0,"tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"RaceDesc","scale":{"type":"linear","zero":false}},"y":{"axis":{"title":"Ratings"},"sort":null,"type":"quantitative","field":"Average","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

In [55]:
DeepnoteChart(Races_mean, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#4c78a8","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"RaceDesc","scale":{"type":"linear"}},"y":{"axis":{"format":".0%"},"sort":null,"type":"quantitative","field":"Termd","scale":{"type":"linear"},"format":{"type":"percent","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"Termd"},"xOffset":{"datum":"Termd"}}}]},{"layer":[{"mark":{"clip":true,"type":"trail","color":"#72b7b2","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"RaceDesc","scale":{"type":"linear","zero":false}},"y":{"axis":{"title":null},"sort":null,"type":"quantitative","field":"PsySafe","scale":{"type":"linear","zero":false,"domainMax":4,"domainMin":1.5},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"}}},{"mark":{"size":100,"type":"point","opacity":0,"tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"RaceDesc","scale":{"type":"linear","zero":false}},"y":{"axis":{"title":null},"sort":null,"type":"quantitative","field":"PsySafe","scale":{"type":"linear","zero":false,"domainMax":4,"domainMin":1.5},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"}}}]},{"layer":[{"mark":{"clip":true,"type":"trail","color":"#e45756","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"RaceDesc","scale":{"type":"linear","zero":false}},"y":{"axis":{"title":null},"sort":null,"type":"quantitative","field":"Voice","scale":{"type":"linear","zero":false,"domainMax":4,"domainMin":1.5},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"Voice"}}},{"mark":{"size":100,"type":"point","opacity":0,"tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"RaceDesc","scale":{"type":"linear","zero":false}},"y":{"axis":{"title":null},"sort":null,"type":"quantitative","field":"Voice","scale":{"type":"linear","zero":false,"domainMax":4,"domainMin":1.5},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"Voice"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#72b7b2","tooltip":true},"encoding":{"x":{"sort":null,"scale":{"type":"linear"}},"y":{"sort":null,"scale":{"type":"linear"}}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","resolve":{"scale":{"y":"independent"}},"encoding":{}}""")

1.White people take up the most percentage in the company, Hispanic the least.

2.We could find that there are two races where no people leave.

3.After evaluating the features, we could find that, American Indian and Hispanic tend to have higher PsySafe, which relate to the  lower Termination Rate as well.

4.But the share of race in a firm's total headcount doesn't seem to affect key characteristics.  For example, according to common sense, the smaller the proportion of race in the company's headcount, the easier it is to be ignored.  However, Hispanic’s voice average is the highest, indicating that the company may be deliberately taking care of minority groups, although their proportion is very small.

5.American Indian and Hispanic make up only a tiny fraction of the company's headcount. Instead, whites and African-Americans, who make up the majority, have higher turnover rates. 

Conclusion: Although race accounts for different proportions of the company's headcount, the company seems to have paid attention to racial diversity and pay attention to minorities.

- For citizens

In [56]:
Citizen = df_environment[['CitizenDesc','Trust','OCB', 'Voice','EmpSatisfaction','PsySafe','OrgIden','TeamIden','Employee.Name','*VoluntaryLeft','Termd']].groupby("CitizenDesc")
Citizen_mean = Citizen.mean().round(2)
Citizen_mean['Average']=Citizen_mean.iloc[: , :-1].mean(axis=1).round(2)
Citizen_mean
Citizen_mean.reset_index(inplace=True)
Citizen_mean

,CitizenDesc,Trust,OCB,Voice,EmpSatisfaction,PsySafe,OrgIden,TeamIden,*VoluntaryLeft,Termd,Average
0,Eligible NonCitizen,3.17,2.67,3.58,4.08,2.58,2.75,2.83,0.42,0.42,2.76
1,Non-Citizen,2.75,2.50,3.00,4.50,2.00,2.75,3.25,0.75,0.75,2.69
2,US Citizen,3.29,2.93,3.07,3.88,2.50,2.74,3.15,0.27,0.33,2.73


In [57]:
DeepnoteChart(Citizen_mean, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#4c78a8","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"CitizenDesc","scale":{"type":"linear"}},"y":{"axis":{"title":"Ratings"},"sort":null,"type":"quantitative","field":"Trust","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"Trust"},"xOffset":{"datum":"Trust"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"CitizenDesc","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"OCB","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"OCB"},"xOffset":{"datum":"OCB"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"CitizenDesc","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"Voice","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"Voice"},"xOffset":{"datum":"Voice"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"CitizenDesc","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"EmpSatisfaction","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"EmpSatisfaction"},"xOffset":{"datum":"EmpSatisfaction"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"CitizenDesc","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"PsySafe","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"PsySafe"},"xOffset":{"datum":"PsySafe"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"CitizenDesc","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"OrgIden","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"OrgIden"},"xOffset":{"datum":"OrgIden"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"CitizenDesc","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"TeamIden","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"TeamIden"},"xOffset":{"datum":"TeamIden"}}}]},{"layer":[{"mark":{"clip":true,"type":"trail","color":"#4c78a8","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"CitizenDesc","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"quantitative","field":"Average","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"}}},{"mark":{"dx":0,"dy":-1,"fill":"black","type":"text","align":"center","baseline":"bottom"},"encoding":{"x":{"sort":null,"type":"nominal","field":"CitizenDesc","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"quantitative","field":"Average","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"text":{"sort":null,"type":"quantitative","field":"Average"}}},{"mark":{"size":100,"type":"point","opacity":0,"tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"CitizenDesc","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"quantitative","field":"Average","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

# 

In [58]:
DeepnoteChart(Citizen_mean, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#4c78a8","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"CitizenDesc","scale":{"type":"linear"}},"y":{"axis":{"format":".0%"},"sort":null,"type":"quantitative","field":"Termd","scale":{"type":"linear"},"format":{"type":"percent","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"Termd"},"xOffset":{"datum":"Termd"}}}]},{"layer":[{"mark":{"clip":true,"type":"trail","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"CitizenDesc","scale":{"type":"linear","zero":false}},"y":{"axis":{"title":null},"sort":null,"type":"quantitative","field":"Trust","scale":{"type":"linear","zero":false,"domainMax":5,"domainMin":1},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"Trust"}}},{"mark":{"size":100,"type":"point","opacity":0,"tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"CitizenDesc","scale":{"type":"linear","zero":false}},"y":{"axis":{"title":null},"sort":null,"type":"quantitative","field":"Trust","scale":{"type":"linear","zero":false,"domainMax":5,"domainMin":1},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"Trust"}}}]},{"layer":[{"mark":{"clip":true,"type":"trail","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"CitizenDesc","scale":{"type":"linear","zero":false}},"y":{"axis":{"title":null},"sort":null,"type":"quantitative","field":"OCB","scale":{"type":"linear","zero":false,"domainMax":5,"domainMin":1},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"OCB"}}},{"mark":{"size":100,"type":"point","opacity":0,"tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"CitizenDesc","scale":{"type":"linear","zero":false}},"y":{"axis":{"title":null},"sort":null,"type":"quantitative","field":"OCB","scale":{"type":"linear","zero":false,"domainMax":5,"domainMin":1},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"OCB"}}}]},{"layer":[{"mark":{"clip":true,"type":"trail","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"CitizenDesc","scale":{"type":"linear","zero":false}},"y":{"axis":{"title":null},"sort":null,"type":"quantitative","field":"EmpSatisfaction","scale":{"type":"linear","zero":false,"domainMax":5,"domainMin":1},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"EmpSatisfaction"}}},{"mark":{"size":100,"type":"point","opacity":0,"tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"CitizenDesc","scale":{"type":"linear","zero":false}},"y":{"axis":{"title":null},"sort":null,"type":"quantitative","field":"EmpSatisfaction","scale":{"type":"linear","zero":false,"domainMax":5,"domainMin":1},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"EmpSatisfaction"}}}]},{"layer":[{"mark":{"clip":true,"type":"trail","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"CitizenDesc","scale":{"type":"linear","zero":false}},"y":{"axis":{"title":null},"sort":null,"type":"quantitative","field":"PsySafe","scale":{"type":"linear","zero":false,"domainMax":5,"domainMin":1},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"PsySafe"}}},{"mark":{"size":100,"type":"point","opacity":0,"tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"CitizenDesc","scale":{"type":"linear","zero":false}},"y":{"axis":{"title":null},"sort":null,"type":"quantitative","field":"PsySafe","scale":{"type":"linear","zero":false,"domainMax":5,"domainMin":1},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"PsySafe"}}}]},{"layer":[{"mark":{"clip":true,"type":"trail","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"CitizenDesc","scale":{"type":"linear","zero":false}},"y":{"axis":{"title":null},"sort":null,"type":"quantitative","field":"TeamIden","scale":{"type":"linear","zero":false,"domainMax":5,"domainMin":1},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"TeamIden"}}},{"mark":{"size":100,"type":"point","opacity":0,"tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"CitizenDesc","scale":{"type":"linear","zero":false}},"y":{"axis":{"title":null},"sort":null,"type":"quantitative","field":"TeamIden","scale":{"type":"linear","zero":false,"domainMax":5,"domainMin":1},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"TeamIden"}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","resolve":{"scale":{"y":"independent"}},"encoding":{}}""")

I think this result can be combined with the policy of USA's working visa policy (OPT and H1B).

The Non-Citizen has higher value in Team-indentity and Employee-satifaction, however they leave more, this could be because of visa or immigration issues and they had to leave. (For example, the salary/position of the company does not meet the immigration visa standards).

Moreover, the OCB, PsySafe, and Trust of non-citizens are significantly lower than those of citizens and eligible non-citizens.  Non-citizens may have to bear greater psychological pressure, which leads to their low self-esteem (low Trust) and only want to improve their personal abilities, while ignoring the contribution to society (low OCB).

Conclusion: If we want to maintain the non-citizen talent, give them more care?

# Feature related to Career/Reasons why they leave

## 

- [ ] 

## Import dataset

In [59]:
df_1= df[df["TermReason"].str.contains("unhappy|Another|more money|career|hours|N/A-StillEmployed")]

In [60]:
df_2= df[df["TermReason"].str.contains("unhappy|Another|more money|career|hours")]
df_2['*Talent'].value_counts()

0    60
1     2
Name: *Talent, dtype: int64

In [61]:
df_1["TermReason"].value_counts()

N/A-StillEmployed    207
Another position      20
unhappy               14
more money            11
career change          9
hours                  8
Name: TermReason, dtype: int64

In [62]:
df_1_dummies = pd.get_dummies(df_1, prefix=['Department','RaceDesc','ManagerID'], columns=['Department','RaceDesc','ManagerID'])

In [63]:
df_1_dummies.columns

Index(['Employee.Name', 'PerfScore', 'FromDiversityJobFair', 'Salary', 'Termd',
       'PositionID', 'Position', 'State', 'Zip', 'DOB', 'Sex', 'MaritalDesc',
       'CitizenDesc', 'HispanicLatino', 'DateofHire', 'DateofTermination',
       'TermReason', 'EmploymentStatus', 'ManagerName', 'RecruitmentSource',
       'EngagementSurvey', 'EmpSatisfaction', 'SpecialProjectsCount', 'ManPos',
       'TechLev', 'JobStr', 'AssColl', 'AssSelf', 'AssLead', 'TeamIden',
       'OrgIden', 'ConHour', 'CarOpp', 'PsySafe', 'Voice', 'Trust', 'OCB',
       '*VoluntaryLeft', '*LastWorkingDate', '*MonthsSpent', '*DateDiff',
       '*TalentScore', '*Talent', 'Department_Admin Offices',
       'Department_Executive Office', 'Department_IT/IS',
       'Department_Production       ', 'Department_Sales',
       'Department_Software Engineering',
       'RaceDesc_American Indian or Alaska Native', 'RaceDesc_Asian',
       'RaceDesc_Black or African American', 'RaceDesc_Hispanic',
       'RaceDesc_Two or more ra

In [64]:
cols_1_d = ['PerfScore', 'FromDiversityJobFair', 'Salary', 
        'EngagementSurvey', 'EmpSatisfaction', 'SpecialProjectsCount', 'ManPos',
       'TechLev', 'JobStr', 'AssColl', 'AssSelf', 'AssLead', 'TeamIden',
       'OrgIden', 'ConHour', 'CarOpp', 'PsySafe', 'Voice', 'Trust', 'OCB',
       'Department_Admin Offices', 'Department_Executive Office',
       'Department_IT/IS', 'Department_Production       ', 'Department_Sales',
       'Department_Software Engineering',
       'RaceDesc_American Indian or Alaska Native', 'RaceDesc_Asian',
       'RaceDesc_Black or African American', 'RaceDesc_Hispanic',
       'RaceDesc_Two or more races', 'RaceDesc_White', 'ManagerID_1',
       'ManagerID_2', 'ManagerID_3', 'ManagerID_4', 'ManagerID_5',
       'ManagerID_6', 'ManagerID_7', 'ManagerID_9', 'ManagerID_10',
       'ManagerID_11', 'ManagerID_12', 'ManagerID_13', 'ManagerID_14',
       'ManagerID_15', 'ManagerID_16', 'ManagerID_17', 'ManagerID_18',
       'ManagerID_19', 'ManagerID_20', 'ManagerID_21', 'ManagerID_22',
       'ManagerID_39']


cols_1 = ['PerfScore', 'FromDiversityJobFair', 'Salary', 
        'EngagementSurvey', 'EmpSatisfaction', 'SpecialProjectsCount', 'ManPos',
       'TechLev', 'JobStr', 'AssColl', 'AssSelf', 'AssLead', 'TeamIden',
       'OrgIden', 'ConHour', 'CarOpp', 'PsySafe', 'Voice', 'Trust', 'OCB']

- Logistic Regression

In [65]:
# Target Variable: TermedReasons (Dummies)
X_1_d = df_1_dummies[cols_1_d]
y_1_d = df_1_dummies["TermReason"]


X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_1_d, y_1_d, test_size=0.2, random_state=100)


param_grid = {'logisticregression__C': np.logspace(-4,2,150)}


LR_1_d = MODEL(X_train_d, y_train_d, X_test_d, y_test_d,LogisticRegression(solver='lbfgs',max_iter = 10e6, random_state = 0,multi_class='multinomial'),param_grid,MinMaxScaler())

Results using MinMaxScaler():
Best C:  1.8554804201388422
Train score: 0.8651162790697674
Test score / Accuracy: 0.8703703703703703
                 Micro-Averaged          Macro-Averaged
F1             0.8703703703703703      0.3573232323232323  
Precision      0.8703703703703703      0.4669811320754717
Recall         0.8703703703703703      0.3333333333333333

----------------------------------------------

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [66]:
X_1 = df_1_dummies[cols_1]
y_1 = df_1_dummies["TermReason"]


X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size=0.2, random_state=100)


param_grid = {'logisticregression__C': np.logspace(-4,2,150)}

LR_1 = MODEL(X_train_1, y_train_1, X_test_1, y_test_1,LogisticRegression(solver='lbfgs',max_iter = 10e6, random_state = 0,multi_class='multinomial'),param_grid,MinMaxScaler())

Results using MinMaxScaler():
Best C:  2.035751822261104
Train score: 0.8465116279069768
Test score / Accuracy: 0.8703703703703703
                 Micro-Averaged          Macro-Averaged
F1             0.8703703703703703      0.26969072164948454  
Precision      0.8703703703703703      0.2803921568627451
Recall         0.8703703703703703      0.26666666666666666

----------------------------------------------

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

- Feature importance

In [67]:
features, numbers, coefs = sig_features(LR_1, 15)
imp_feats_r= pd.DataFrame(list(zip(features, coefs)),columns=['Feature','Coefficients'] )
imp_feats_r

,Feature,Coefficients
0,Voice,0.3433
1,TechLev,-0.3824
2,OCB,-0.3935
3,Trust,-0.4972
4,AssSelf,-0.5311
5,PerfScore,0.6138
6,ConHour,-0.7202
7,OrgIden,-0.7670
8,JobStr,-0.8166
9,PsySafe,-0.9414


- [ ] OVEARALL

In [68]:
DeepnoteChart(imp_feats_r, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"quantitative","field":"Coefficients","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"y":{"sort":null,"type":"nominal","field":"Feature","scale":{"type":"linear"}}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

In [69]:
scaler = MinMaxScaler()


d_scaled = scaler.fit_transform(X_train_1)
lr = LogisticRegression(multi_class='multinomial', solver='lbfgs',C= 2.035751822261104)
lr.fit(d_scaled, y_train_1)

LogisticRegression(C=2.035751822261104, multi_class='multinomial')

In [70]:

coef = lr.coef_


coef_df = pd.DataFrame(coef, columns=X_train_1.columns)
n_features = 10
top_features = {}
for i, label in enumerate(lr.classes_):
    top_features[label] = dict(coef_df.iloc[i].nlargest(n_features).to_dict())

result_df = pd.DataFrame.from_dict(top_features, orient='index')


result_df = result_df.T


result_df = result_df.stack().reset_index()
result_df.columns = ['feature', 'class', 'coefficient']


result_df = result_df.sort_values(['class', 'coefficient'], ascending=[True, False])


result_df = result_df.groupby('class').head(n_features)

In [71]:
result_df

,feature,class,coefficient
0,EngagementSurvey,Another position,1.485454
4,FromDiversityJobFair,Another position,1.386688
8,EmpSatisfaction,Another position,1.014961
13,PerfScore,Another position,0.613831
16,Voice,Another position,0.343345
19,Salary,Another position,0.261530
22,AssLead,Another position,0.135794
25,AssColl,Another position,0.129064
28,SpecialProjectsCount,Another position,0.051258
31,ManPos,Another position,-0.172783


- Important features for Happy

In [72]:
df_h = result_df[result_df["class"].str.contains("unhappy")]

In [73]:
DeepnoteChart(df_h, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"quantitative","field":"coefficient","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"y":{"sort":null,"type":"nominal","field":"feature","scale":{"type":"linear"}}}}]}],"title":"Important Features related to Unhappy","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

- [ ] For no reason

In [74]:
df_N = result_df[result_df["class"].str.contains("N/A")]

In [75]:
DeepnoteChart(df_N, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"quantitative","field":"coefficient","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"y":{"sort":null,"type":"nominal","field":"feature","scale":{"type":"linear"}}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

In [76]:
df_hour = result_df[result_df["class"].str.contains("hours")]

In [77]:
DeepnoteChart(df_hour, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"quantitative","field":"coefficient","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"y":{"sort":null,"type":"nominal","field":"feature","scale":{"type":"linear"}}}}]}],"title":"Important Features Related to Hour","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

- [ ] career

In [78]:
df_career = result_df[result_df["class"].str.contains("career")]

In [79]:
DeepnoteChart(df_career, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"quantitative","field":"coefficient","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"y":{"sort":null,"type":"nominal","field":"feature","scale":{"type":"linear"}}}}]}],"title":"Important Features Related to career change","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

In [80]:
df_money = result_df[result_df["class"].str.contains("money")]

In [81]:
DeepnoteChart(df_money, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"quantitative","field":"coefficient","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"y":{"sort":null,"type":"nominal","field":"feature","scale":{"type":"linear"}}}}]}],"title":"Important Features Related to more money","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

In [82]:
df_another = result_df[result_df["class"].str.contains("position")]

In [118]:
DeepnoteChart(df_another, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"quantitative","field":"coefficient","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"y":{"sort":null,"type":"nominal","field":"feature","scale":{"type":"linear"}}}}]}],"title":"Important Features Related to Another station","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

## basic data analysis (talent percentage per reason ;caree_socre)

### The Reason per Talent Leave

In [84]:
df_reason_ratio = df_1[['TermReason','*Talent',"*DateDiff"]].groupby("TermReason")

In [85]:
df_reason_ratio_mean = df_reason_ratio.mean().round(2)
df_reason_ratio_mean
df_reason_ratio_mean.reset_index(inplace=True)
df_reason_ratio_mean

,TermReason,*Talent,*DateDiff
0,Another position,0.05,8.65
1,N/A-StillEmployed,0.31,8.59
2,career change,0.00,9.78
3,hours,0.12,10.25
4,more money,0.00,8.73
5,unhappy,0.00,8.21


### 

In [86]:
df_2= df[df["TermReason"].str.contains("unhappy|Another|more money|career|hours")]

## Suggestion : Simulation to see how to  improve the condition

### Working evirnoment

- [ ] key Feature improved by 5%

In [87]:
df_imp_result = pd.DataFrame(columns=['Improvement', 'Termed', 'Talent'])

In [88]:
df_alternative = df

In [89]:
df_store = pd.DataFrame()

In [90]:
columns_to_increase = ['PsySafe', 'Trust','Voice','OCB','EmpSatisfaction','OrgIden','TeamIden']

df_alternative[columns_to_increase] *= 1.05

In [91]:
X_improved_e = df_alternative[cols]

In [92]:
df_store['5%'] = LR_3.predict(X_improved_e)

- [ ] Key feature improved by 10%

In [93]:
df_alternative = df
df_alternative[columns_to_increase] *= 1.1
X_improved_e = df_alternative[cols]
df_store['10%']  = LR_3.predict(X_improved_e)


- [ ] improved by 15%

In [94]:
df_alternative = df
df_alternative[columns_to_increase] *= 1.15
X_improved_e = df_alternative[cols]
df_store['15%'] = LR_3.predict(X_improved_e)


- [ ] improved by 20%

In [95]:
df_alternative = df
df_alternative[columns_to_increase] *= 1.2
X_improved_e = df_alternative[cols]
df_store['20%'] = LR_3.predict(X_improved_e)

In [96]:
df_store.sum()

5%     79
10%    67
15%    53
20%    35
dtype: int64

In [97]:
df_imp_result['Improvement'] = ['None','5%','10%','15%','20%']
df_imp_result['Termed'] = [88,79,67,53,35]
df_imp_result['Talent'] = [2,1,0,0,0]

In [98]:
DeepnoteChart(df_imp_result, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","tooltip":true},"encoding":{"x":{"axis":{"title":"Count"},"sort":null,"type":"quantitative","field":"Termed","scale":{"type":"linear"},"stack":"zero","format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"y":{"sort":null,"type":"nominal","field":"Improvement","scale":{"type":"linear"},"stack":"zero"},"color":{"sort":null,"type":"quantitative","field":"Termed","scale":{"scheme":"blueorange"}}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","tooltip":true},"encoding":{"x":{"sort":null,"type":"quantitative","field":"Talent","scale":{"type":"linear"},"stack":"zero","format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"y":{"sort":null,"type":"nominal","field":"Improvement","scale":{"type":"linear"},"stack":"zero"},"color":{"sort":null,"type":"quantitative","field":"Talent","scale":{"scheme":"blueorange"}}}}]}],"title":"Simulation of feature improvement with number of people Voluntarily Termed","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

### career opportunities and Reasons

In [99]:
col_career_to_decreas =['ConHour','JobStr']
col_career_to_increas =['PsySafe','CarOpp','Trust','EngagementSurvey','Salary']



In [100]:
df_store_3 = pd.DataFrame()

In [101]:
df_alternative = df_1_dummies
df_alternative[col_career_to_increas] *= 1.05
df_alternative[col_career_to_decreas] *= 0.95
X_improved_career = df_alternative[cols_1]
df_store_3['5%']  = LR_1.predict(X_improved_career)

In [102]:
df_alternative = df_1_dummies
df_alternative[col_career_to_increas] *= 1.1
df_alternative[col_career_to_decreas] *= 0.9
X_improved_career = df_alternative[cols_1]
df_store_3['10%']  = LR_1.predict(X_improved_career)

In [103]:
df_alternative = df_1_dummies
df_alternative[col_career_to_increas] *= 1.15
df_alternative[col_career_to_decreas] *= 0.85
X_improved_career = df_alternative[cols_1]
df_store_3['15%']  = LR_1.predict(X_improved_career)

In [104]:
df_alternative = df_1_dummies
df_alternative[col_career_to_increas] *= 1.2
df_alternative[col_career_to_decreas] *= 0.8
X_improved_career = df_alternative[cols_1]
df_store_3['20%']  = LR_1.predict(X_improved_career)

In [105]:
df_store_3['5%'].value_counts()

N/A-StillEmployed    234
Another position      19
unhappy                6
career change          6
more money             4
Name: 5%, dtype: int64

In [106]:
df_store_3['10%'].value_counts()

N/A-StillEmployed    236
Another position      25
career change          4
unhappy                3
more money             1
Name: 10%, dtype: int64

In [107]:
df_store_3['15%'].value_counts()

N/A-StillEmployed    236
Another position      30
career change          2
unhappy                1
Name: 15%, dtype: int64

In [108]:
df_store_3['20%'].value_counts()

N/A-StillEmployed    235
Another position      33
career change          1
Name: 20%, dtype: int64

In [109]:
df_imp_result_1 = pd.DataFrame(columns=['Improvement', 'Reeason','Count'])
df_imp_result_1['Improvement'] =['None','None','None','None',
    '5%','5%','5%','5%',
    '10%','10%','10%','10%',
    '15%','15%','15%','15%',
    '20%','20%','20%','20%']

df_imp_result_1['Reason'] =['Unhappy','More money','Career change','Hours',
'Unhappy','More money','Career change','Hours',
'Unhappy','More money','Career change','Hours',
'Unhappy','More money','Career change','Hours',
'Unhappy','More money','Career change','Hours']

df_imp_result_1['Count'] = [14,11,9,8,
6,4,6,0,
3,1,4,0,
1,0,2,0,
0,0,1,0]

In [110]:
df_imp_result_1_group =  df_imp_result_1.groupby('Improvement')

In [111]:
df_imp_result_1.to_excel('output.xlsx', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=05534a36-b3b0-4f79-89b0-0c5bea521dde' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>